In [1]:
import json
import os
import gc
from types import SimpleNamespace
import pathlib
import numpy as np
from PIL import Image
import tensorflow as tf

In [2]:
# Define the directory of the dataset
data_dir = pathlib.Path('CatsDogs/')

# Remove corrupted files
os.system("rm CatsDogs/Cats/666.jpg CatsDogs/Dogs/11702.jpg CatsDogs/Dogs/11410.jpg")

# Collects the path of all the files within the dataset
data_paths = [str(path) for path in list(data_dir.glob("*/*.jpg"))]
print(f"Images in the dataset: {len(data_paths)}")

rm: cannot remove 'CatsDogs/Cats/666.jpg': No such file or directory
rm: cannot remove 'CatsDogs/Dogs/11702.jpg': No such file or directory
rm: cannot remove 'CatsDogs/Dogs/11410.jpg': No such file or directory


Images in the dataset: 24997


In [3]:
# Convert non-jpeg images into jpeg files
formats = [(path, Image.open(path).format) for path in data_paths]
non_jpegs = list(filter(lambda x: x[1]!='JPEG', formats))
for path, _ in non_jpegs:
    img = Image.open(path)
    img.convert('RGB').save(path, format='JPEG')

"# Convert non-jpeg images into jpeg files\nformats = [(path, Image.open(path).format) for path in data_paths]\nnon_jpegs = list(filter(lambda x: x[1]!='JPEG', formats))\nfor path, _ in non_jpegs:\n    img = Image.open(path)\n    img.convert('RGB').save(path, format='JPEG')"

In [4]:
# Get the class names
class_names = np.array(sorted([item.name for item in data_dir.glob('*') if not item.name.startswith('.')]))
print(class_names)

['Cats' 'Dogs']


In [5]:
def get_label(file_path):
    # Convert the path to a list of path components
    parts = tf.strings.split(file_path, os.path.sep)
    # The second to last is the class-directory
    one_hot = parts[-2] == class_names
    # Integer encode the label
    return tf.argmax(one_hot)

def decode_img(img):
    # Convert the compressed string to a 3D uint8 tensor
    img = tf.io.decode_jpeg(img, channels=3)
    # Resize the image to the desired size
    return tf.image.resize(img, [img_height, img_width])

def process_path(file_path):
    label = get_label(file_path)
    # Load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

# Configure dataset for performance
def configure_for_performance(ds):
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    return ds

def zero_one_loss(dataset, dataset_size):
    
    _, accuracy = model.evaluate(dataset)
    zero_one_loss = dataset_size*(1-accuracy)

    return int(round(zero_one_loss, ndigits=0))

In [6]:
# Set initial params for the loader
batch_size = 64
img_height = 256
img_width = 256

# Load hyper-parameters
with open('best_model_results.json') as f:
    best_model = json.load(f)


hyper_parameters = SimpleNamespace(**best_model)

# K-fold cross validation

In [7]:
from sklearn.model_selection import KFold


k_fold = KFold(n_splits=5, shuffle=True)
k_splits = k_fold.split(data_paths)
results = []

for train_index, test_index in k_splits:

    # Get the paths to the data
    train_paths = np.asarray(data_paths)[train_index]
    test_paths = np.asarray(data_paths)[test_index]

    # Make it tf.data.Dataset
    train = tf.data.Dataset.from_tensor_slices(train_paths)
    test = tf.data.Dataset.from_tensor_slices(test_paths)

    # Shuffle the dataset
    train = train.shuffle(len(train))
    test = test.shuffle(len(test))

    # Get labels
    train = train.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
    test = test.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
    
    # Configure for performance
    train = configure_for_performance(train)
    test = configure_for_performance(test)

    # Create the model
    model = tf.keras.Sequential([
        tf.keras.layers.Rescaling(1./255),
        tf.keras.layers.Conv2D(hyper_parameters.n_filters, hyper_parameters.kernel_size, activation=tf.nn.relu, input_shape=(img_height, img_width, 3)),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(hyper_parameters.n_filters * (1, 2)[hyper_parameters.filters_coeff=='incremental'], hyper_parameters.kernel_size, activation=tf.nn.relu),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(hyper_parameters.n_filters * (1, 4)[hyper_parameters.filters_coeff=='incremental'], hyper_parameters.kernel_size, activation=tf.nn.relu),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation=tf.nn.relu),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    model.fit(
        train,
        epochs=hyper_parameters.n_epochs,
        verbose=0,
    )

    loss = zero_one_loss(test, len(test_paths))
    results.append(loss)
    print(f"Zero-one loss: {loss}")

    # Clear any previous state
    del model
    tf.keras.backend.clear_session()
    gc.collect()

2022-07-23 12:29:06.391655: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-23 12:29:06.547059: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-23 12:29:06.548180: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-23 12:29:06.551378: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

18/79 [=====>........................] - ETA: 20s - loss: 1.6201 - accuracy: 0.7960

Corrupt JPEG data: 2226 extraneous bytes before marker 0xd9


28/79 [=========>....................] - ETA: 16s - loss: 1.5491 - accuracy: 0.8086

Corrupt JPEG data: 128 extraneous bytes before marker 0xd9


...

40/79 [==============>...............] - ETA: 6s - loss: 2.0761 - accuracy: 0.7309

Corrupt JPEG data: 214 extraneous bytes before marker 0xd9


50/79 [=================>............] - ETA: 4s - loss: 1.9654 - accuracy: 0.7350

Corrupt JPEG data: 396 extraneous bytes before marker 0xd9


79/79 [==============================] - 12s 131ms/step - loss: 1.9263 - accuracy: 0.7379
Zero-one loss: 1310


In [8]:
mean_loss = np.round(np.mean(results), decimals=0)
std_loss = np.round(np.std(results), decimals=0)
print(f'The mean of zero-one loss is {int(mean_loss)}, with a standard deviation of {int(std_loss)} missmatched samples')

The mean of zero-one loss is 1089, with a standard deviation of 113 missmatched samples
